Tutorial with  first examples of how setting up an espresso  calculator and perform a Total Energy and bast structure calculations. 

1. Setting up the system.  

In [196]:
from ase import Atoms
from ase.lattice import FCC
from ase.visualize import  view

si_fcc = Atoms(symbols='Si2',
               cell = FCC(a=10.20 * 0.529177).tocell(),
               scaled_positions=[(0.0, 0.0, 0.0),
                                 (0.25, 0.25, 0.25)],
               pbc=True)


setup basic PW calculator for scf calculation:

1. The main input variables: 

In [197]:
control = {'calculation': "scf",
           'prefix': "si_fcc",
           'pseudo_dir': "qe-7.2/pseudo",
           'tprnfor': True,
           'tstress': True}
system = {'ecutwfc': 35.00}
electrons = {'diagonalization':'davidson',
             'conv_thr': 1.e-8}

2. The pseudopotentials for each atomic species
2. The MP mesh for kpoints 
2. The offset of the MP mesh 
2. The command for starting the calculations 

In [198]:
atomic_species = {'Si':'Si.pz-vbc.UPF'}
kpts = [8,8,8] #8X8X8 MP k-point mesh 
koffset = [0,0,0] 


6. set pw command. For the correct interaction with the ase calculator:
    * the input file must be `{label}.pwi`
    * the output file must be `{label}.pwo` 

In [199]:

def pwcommand (label='scf'): 
    return f"mpirun -np 4 /qe-7.2/bin/pw.x -i {label}.pwi > {label}.pwo" 

In [200]:
pwcommand('scf')

'mpirun -np 4 /scratch/pdelugas/build_intel/bin/pw.x -i scf.pwi > scf.pwo'

In [201]:
from ase.calculators.espresso import Espresso as Qepw
scf_calc = QEpw(label='scf',
                command = pwcommand('scf'),
                input_data={'control': control,
                            'system': system,
                            'electrons': electrons},
                pseudopotentials=atomic_species,
                kpts=[8,8,8],
                koffset=[1,1,1])


In [205]:
si_fcc.calc = scf_calc
si_fcc.get_properties(['energy'])

(Properties({'energy': -215.69011555407923, 'free_energy': -215.69011555407923, 'natoms': 2, 'forces': array([[-0.,  0.,  0.],
       [ 0., -0., -0.]]), 'stress': array([-0.00065006, -0.00065006, -0.00065006,  0.        ,  0.        ,
       -0.        ])})

Setup the calculator for computing the bands. 
1. First we generate the k-point path using ASE and its default settings:

In [206]:
kpointpath = si_fcc.cell.bandpath()

BandPath(path='GXWKGLUWLK,UX', cell=[3x3], special_points={GKLUWX}, kpts=[37x3])

2. Then we setup the new calculator for bands:

In [207]:
control.update({'calculation':"bands"})
bands_calc = QEpw(label='bands',
                  input_data={'control': control,
                              'system': system,
                              'electrons':electrons},
                  pseudopotentials=atomic_species, 
                  command=pwcommand(label='bands'),
                  kpts=kpointpath) 


['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_deprecated',
 '_deprecated_get_spin_polarized',
 '_directory',
 'atoms',
 'band_structure',
 'calc',
 'calculate',
 'calculate_numerical_forces',
 'calculate_numerical_stress',
 'calculate_properties',
 'calculation_required',
 'check_state',
 'command',
 'default_parameters',
 'directory',
 'discard_results_on_any_change',
 'export_properties',
 'get_atoms',
 'get_charges',
 'get_default_parameters',
 'get_dipole_moment',
 'get_eigenvalues',
 'get_fermi_level',
 'get_forces',
 'get_ibz_k_points',
 'get_k_point_weights',
 'get_magnetic_moment',
 'ge